In [3]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
 
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
 
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini
 
g = list()
# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    g.append(b_score)
    return {'index':b_index, 'value':b_value, 'groups':b_groups, 'gini_index':b_score}
 
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    #print(root)
    return root
 
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
 
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample
''' 
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    temp = predictions
    print(temp)
    #------------------------------------ Code for adding Weight for all the trees ----------------------------------
    w_zero = 0
    w_one = 0
    weight = [0.2, 0.3, 0.6, 0.5, 0.8]
    for i in [0,1,2,3,4]:
        if temp[i] == 0:
            w_zero += weight[i]
        else:
            w_one += weight[i]
    
    print("Weight Zero = ", w_zero)
    print("Weight One = ", w_one)
    
    print(max(set(predictions), key=predictions.count))
    
    if w_zero > w_one :
        print("Weighted Prediction =  0")
        return 0
    else:
        print("Weighted Prediction =  1")
        return 1
    
    
    #----------------------------------------------------------------------------------------------------------------
    #print(predictions)
    #print(max(set(predictions), key=predictions.count))
    #return max(set(predictions), key=predictions.count)
 '''   
'''
#--------------------------------------Fuction for calculating Weight of trees-----------------------------------------
def calclate_weight(trees):
    weight = [c_weight(tree) for tree in trees]
    return weight

def c_weight(tree):
    return 1
    
def calulate_weight(tree):
    weight = root
    return weight
    




#----------------------------------------------------------------------------------------------------------------------
'''
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row, n_trees, tree_weight):
    predictions = [predict(tree, row) for tree in trees]
    temp = predictions
    #print(temp)
    #------------------------------------ Code for adding Weight for all the trees ----------------------------------
    w_zero = 0
    w_one = 0
    weight = [0.2, 0.3, 0.6, 0.5, 0.8]
    for i in range(n_trees):
        if temp[i] == 0:
            #w_zero += weight[i]
            w_zero += tree_weight[i]
        else:
           # w_one += weight[i]
            w_one += tree_weight[i]
    
    #print("Weight Zero = ", w_zero)
    #print("Weight One = ", w_one)
    
    #print(max(set(predictions), key=predictions.count))
    
    if w_zero > w_one :
        #print("Weighted Prediction =  0")
        return 0
    else:
        #print("Weighted Prediction =  1")
        return 1
    
    
    #----------------------------------------------------------------------------------------------------------------
    #print(predictions)
    #print(max(set(predictions), key=predictions.count))
    #return max(set(predictions), key=predictions.count)
    
#----------------------------------------------sum gini index --------------------------------------------------------    
def sum_gini(tree):
    tree['gini_index']
    sum = 0.0
    sum += tree['gini_index']
    return sum

#---------------------------------------------------------------------------------------------------------------------

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    weght = list() 
    tree_weight = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
        
        print(tree)
        #print('\n\n')
#-----------------------------------------------------Weight Allocation-----------------------------------------------
        y = sum_gini(tree)
        #print(y)
        #x = calulate_weight(tree)
        #x = (i+1) / n_trees
        #tree_weight.append(x)  
        tree_weight.append(y)  
        #print(tree_weight)
    #tree_weight = [calclate_weight(trees)]
#---------------------------------------------------------------------------------------------------------------------        
    #print(trees)
    predictions = [bagging_predict(trees, row, n_trees, tree_weight) for row in test]
    #print(predictions)
    return(predictions)
 
# Test the random forest algorithm
seed(2)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 2
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [5, 10, 20]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

{'index': 45, 'value': 0.1952, 'gini_index': 0.3415948275862069, 'left': {'index': 10, 'value': 0.1995, 'gini_index': 0.32511434297754255, 'left': 0, 'right': 1}, 'right': {'index': 9, 'value': 0.0998, 'gini_index': 0.07978723404255299, 'left': 0, 'right': 1}}
{'index': 50, 'value': 0.0129, 'gini_index': 0.3618795177218319, 'left': {'index': 47, 'value': 0.0772, 'gini_index': 0.32606837606837613, 'left': 0, 'right': 0}, 'right': {'index': 44, 'value': 0.196, 'gini_index': 0.26192993053458175, 'left': 1, 'right': 1}}
{'index': 10, 'value': 0.221, 'gini_index': 0.39058168693985607, 'left': {'index': 24, 'value': 0.578, 'gini_index': 0.246008502598016, 'left': 1, 'right': 0}, 'right': {'index': 25, 'value': 0.7256, 'gini_index': 0.32854488506662416, 'left': 0, 'right': 1}}
{'index': 3, 'value': 0.0376, 'gini_index': 0.41501966126089074, 'left': {'index': 43, 'value': 0.3387, 'gini_index': 0.1922023182297156, 'left': 0, 'right': 1}, 'right': {'index': 45, 'value': 0.0798, 'gini_index': 0.3

{'index': 45, 'value': 0.0691, 'gini_index': 0.405310742183052, 'left': {'index': 12, 'value': 0.299, 'gini_index': 0.20933333333333332, 'left': 0, 'right': 1}, 'right': {'index': 7, 'value': 0.0333, 'gini_index': 0.3373138729175882, 'left': 0, 'right': 1}}
{'index': 58, 'value': 0.011, 'gini_index': 0.43489735041235306, 'left': {'index': 51, 'value': 0.0054, 'gini_index': 0.40893600908746697, 'left': 0, 'right': 0}, 'right': {'index': 3, 'value': 0.0566, 'gini_index': 0.06, 'left': 0, 'right': 1}}
{'index': 19, 'value': 0.5167, 'gini_index': 0.42695744525012813, 'left': {'index': 9, 'value': 0.1533, 'gini_index': 0.3581466842336407, 'left': 0, 'right': 1}, 'right': {'index': 2, 'value': 0.0115, 'gini_index': 0.3217887396991874, 'left': 0, 'right': 1}}
{'index': 35, 'value': 0.4546, 'gini_index': 0.3623030293412888, 'left': {'index': 10, 'value': 0.1748, 'gini_index': 0.1821913273750415, 'left': 0, 'right': 1}, 'right': {'index': 53, 'value': 0.0234, 'gini_index': 0.2586206896551723, '

{'index': 55, 'value': 0.0053, 'gini_index': 0.4528164924506388, 'left': {'index': 35, 'value': 0.1787, 'gini_index': 0.25987841945288764, 'left': 1, 'right': 0}, 'right': {'index': 51, 'value': 0.0071, 'gini_index': 0.35555555555555546, 'left': 0, 'right': 1}}
{'index': 2, 'value': 0.0257, 'gini_index': 0.4309501474135621, 'left': {'index': 50, 'value': 0.0133, 'gini_index': 0.24230155809103174, 'left': 0, 'right': 1}, 'right': {'index': 30, 'value': 0.4072, 'gini_index': 0.3334263392857143, 'left': 1, 'right': 0}}
{'index': 35, 'value': 0.1975, 'gini_index': 0.4074443690427084, 'left': {'index': 45, 'value': 0.22, 'gini_index': 0.19103343465045586, 'left': 1, 'right': 0}, 'right': {'index': 45, 'value': 0.2216, 'gini_index': 0.27877795962902346, 'left': 0, 'right': 1}}
{'index': 11, 'value': 0.2226, 'gini_index': 0.31808943089430886, 'left': {'index': 16, 'value': 0.6687, 'gini_index': 0.32291666666666663, 'left': 0, 'right': 0}, 'right': {'index': 27, 'value': 0.66, 'gini_index': 0.

{'index': 4, 'value': 0.0497, 'gini_index': 0.38565698495155154, 'left': {'index': 49, 'value': 0.007, 'gini_index': 0.2858999350227421, 'left': 1, 'right': 0}, 'right': {'index': 16, 'value': 0.1647, 'gini_index': 0.34200055519570643, 'left': 0, 'right': 1}}
{'index': 4, 'value': 0.0419, 'gini_index': 0.4216463414634146, 'left': {'index': 27, 'value': 0.7325, 'gini_index': 0.2380952380952381, 'left': 0, 'right': 1}, 'right': {'index': 39, 'value': 0.5018, 'gini_index': 0.38306100217864925, 'left': 1, 'right': 0}}
{'index': 30, 'value': 0.8787, 'gini_index': 0.44688165002455404, 'left': {'index': 27, 'value': 0.9812, 'gini_index': 0.38758389261744963, 'left': 0, 'right': 1}, 'right': {'index': 36, 'value': 0.6241, 'gini_index': 0.0, 'left': 1, 'right': 1}}
{'index': 48, 'value': 0.037, 'gini_index': 0.3622505543237251, 'left': {'index': 18, 'value': 0.5466, 'gini_index': 0.24101479915433413, 'left': 0, 'right': 1}, 'right': {'index': 4, 'value': 0.0393, 'gini_index': 0.2461229946524064

{'index': 35, 'value': 0.4572, 'gini_index': 0.4067618018837531, 'left': {'index': 46, 'value': 0.053, 'gini_index': 0.3217153217153217, 'left': 0, 'right': 1}, 'right': {'index': 43, 'value': 0.4016, 'gini_index': 0.2768194768194769, 'left': 0, 'right': 1}}
{'index': 20, 'value': 0.6508, 'gini_index': 0.4323251741250059, 'left': {'index': 48, 'value': 0.0698, 'gini_index': 0.27003165084124603, 'left': 0, 'right': 1}, 'right': {'index': 43, 'value': 0.145, 'gini_index': 0.2474348346688773, 'left': 0, 'right': 1}}
{'index': 51, 'value': 0.0045, 'gini_index': 0.447652231624357, 'left': {'index': 24, 'value': 0.7745, 'gini_index': 0.0, 'left': 0, 'right': 0}, 'right': {'index': 8, 'value': 0.1618, 'gini_index': 0.35955934612651025, 'left': 0, 'right': 1}}
{'index': 46, 'value': 0.0805, 'gini_index': 0.43847515205760906, 'left': {'index': 27, 'value': 0.9238, 'gini_index': 0.341090909090909, 'left': 0, 'right': 1}, 'right': {'index': 47, 'value': 0.1196, 'gini_index': 0.40563752912218826, 

{'index': 41, 'value': 0.0824, 'gini_index': 0.43290622940013185, 'left': {'index': 42, 'value': 0.1476, 'gini_index': 0.0, 'left': 0, 'right': 1}, 'right': {'index': 11, 'value': 0.2226, 'gini_index': 0.3977684581132857, 'left': 0, 'right': 1}}
{'index': 24, 'value': 0.7973, 'gini_index': 0.44951339852408545, 'left': {'index': 51, 'value': 0.0045, 'gini_index': 0.35352469396369773, 'left': 0, 'right': 1}, 'right': {'index': 34, 'value': 0.236, 'gini_index': 0.2685245901639345, 'left': 1, 'right': 0}}
{'index': 10, 'value': 0.1734, 'gini_index': 0.3122822299651568, 'left': {'index': 31, 'value': 0.1235, 'gini_index': 0.10107816711590299, 'left': 1, 'right': 0}, 'right': {'index': 16, 'value': 0.3512, 'gini_index': 0.3206766917293233, 'left': 1, 'right': 1}}
{'index': 11, 'value': 0.2251, 'gini_index': 0.3310651584776717, 'left': {'index': 21, 'value': 0.7507, 'gini_index': 0.2479049590850833, 'left': 0, 'right': 1}, 'right': {'index': 51, 'value': 0.004, 'gini_index': 0.139641943734015